In [ ]:
!pip install pandas

In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
df_purchases = pd.read_csv("purchases.csv")
df_leads = pd.read_csv("leads.csv")
df_joined = df_leads.set_index("customer_id").join(df_purchases.set_index("customer_id"), on="customer_id", how="left", )

# %%
null_purchase_values = df_purchases.isnull()
null_purchase_counts = null_purchase_values.sum()

print("Null values found in purchases:")
print(null_purchase_counts)

null_leads_values = df_leads.isnull()
null_leads_counts = null_leads_values.sum()

print("Null values found in leads:")
print(null_leads_counts)

## Fill missing gender
filling_gender = df_joined['gender'].mode()[0]
df_joined['gender'].fillna(filling_gender, inplace=True)

print(f"Most common gender: {filling_gender}")

## Fill missing nationality and province
filling_nationality = df_joined['nationality'].mode()[0]
filling_province = df_joined['province'].mode()[0]
df_joined['nationality'].fillna(filling_nationality, inplace=True)
df_joined['province'].fillna(filling_province, inplace=True)


print(f"Most common nationality: {filling_nationality}")
print(f"Most common province: {filling_province}")

df_joined['dob'] = pd.to_datetime(df_joined['dob'], dayfirst=True, format="mixed")
df_joined.head()

df_joined['today'] = datetime.today().strftime("%d/%m/%Y")
df_joined['today'] = pd.to_datetime(df_joined['today'], dayfirst=True, errors='ignore')

df_joined = df_joined.assign(Age=lambda x: round((x['today'] - x['dob']).dt.days))
filling_dob = (datetime.today() - timedelta(days=df_joined['Age'].median())).date()
df_joined['dob'].fillna(filling_dob, inplace=True)


print(f"Most common age: {df_joined['Age'].median()}")
print(f"DOB used: {filling_dob}")
df_joined.drop(columns=['today','Age'], inplace=True)

# Export filled file
df_joined.to_csv("CSV/1_data_cleaning.csv")

In [ ]:
from datetime import datetime
import pandas as pd
df_purchases = pd.read_csv("purchases.csv")
df_leads = pd.read_csv("leads.csv")

df_joined = df_leads.set_index("customer_id").join(df_purchases.set_index("customer_id"), on="customer_id", how="left", )
df_joined.head()
df_joined['booking_date'] = pd.to_datetime(df_joined['booking_date'], dayfirst=True, errors='ignore')
df_joined['lead_date'] = pd.to_datetime(df_joined['lead_date'], dayfirst=True, errors='ignore')
df_joined_modified = df_joined.assign(ConvertedInDays=lambda x: (x['booking_date'] - x['lead_date']).dt.days)

df_joined_modified.head()

df_joined_modified[df_joined_modified['ConvertedInDays'] > 0].head()


df_joined.to_csv("CSV/2_data_manipulation.csv")


In [ ]:
from datetime import datetime
import pandas as pd
import numpy as np

df_purchases = pd.read_csv("purchases.csv")
df_leads = pd.read_csv("leads.csv")
df_joined = df_leads.set_index("customer_id").join(df_purchases.set_index("customer_id"), on="customer_id", how="left", )

# Add flag for converted customers
df_joined['IsEconomy'] = df_joined['product_searched'].isin(['Economy'])
df_joined['BoughtPremiumOrBusiness'] = df_joined['product_booked'].isin(['Premium_Economy','Business'])
df_joined = df_joined.assign(HasUpgraded=lambda x: x['IsEconomy'] & x['BoughtPremiumOrBusiness'])
#df_joined.assign(IsUpgraded=lambda x: True if x['product_searched'].str.contains("Economy") else False)

# df_joined.head()
df_joined.drop(columns=['IsEconomy','BoughtPremiumOrBusiness'], inplace=True)
df_joined[df_joined['HasUpgraded']].head()

# Add Converted target variables
df_joined['Converted'] = df_joined['HasUpgraded']
df_joined = df_joined.assign(HasUpgraded=lambda x: "Converted" if x['Converted'].bool else None)

# Final booking price
df_joined["FinalPrice"] = df_joined["Booking_Price"] - df_joined["Discount"]
df_joined.head()

df_joined.to_csv("CSV/3_data_transformation.csv")